# Bifurcations of the perfect system: sympy analysis

$$\newcommand{\E}{\mathcal{E}}$$

In [ ]:
#import sympy

from sympy import Eq, solve, Symbol, symbols

In [ ]:
def display_dict(d):
    for k, v in d.items():
        display(Eq(k, v))

In [ ]:
λ, u, v = symbols(r"\lambda u v")

In [ ]:
δu, δv, δw = symbols(r"\delta{}u \delta{}v \delta{}w")

In [ ]:
u0_dot, u0_ddot, u0_dddot, u0_ddddot = symbols(r"\dot{u}_0 \ddot{u}_0 \dddot{u}_0 \ddddot{u}_0")

u_star = λ * u0_dot + λ**2 * u0_ddot / 2 + λ**3 * u0_dddot / 6 + λ**4 * u0_ddddot / 24

In [ ]:
E_λ = symbols(r"\E_\lambda")
E2, E_uλ, E_λλ = symbols(r"\E_2 \E_{u\lambda} \E_{\lambda\lambda}")
E3, E_uuλ, E_uλλ, E_λλλ = symbols(r"\E_3 \E_{uu\lambda} \E_{u\lambda\lambda} \E_{\lambda\lambda\lambda}")
E4, E_uuuλ, E_uuλλ, E_uλλλ, E_λλλλ = symbols(r"\E_4 \E_{uuu\lambda} \E_{uu\lambda\lambda}"
                                                 " \E_{u\lambda\lambda\lambda} \E_{\lambda\lambda\lambda\lambda}")

E = (λ * E_λ 
     + (E2 * u**2 + 2 * λ * E_uλ * u + λ**2 * E_λλ) / 2 
     + (E3 * u**3 + 3 * λ * E_uuλ * u**2 + 3 * λ**2  * E_uλλ * u + λ**3 * E_λλλ) / 6
     + (E4 * u**4 + 4 * λ * E_uuuλ * u**3 + 6 * λ**2 * E_uuλλ * u**2 + 4 * λ**3 * E_uλλλ * u + λ**4 * E_λλλλ) / 24)
E

It will be assumed in the following that $v, \delta v \in V$, therefore terms of the form $\E_2 \, v$ and $\E_2 \, \delta v$ should vanish. The following function performs this simplification.

In [ ]:
def simplify_E2(expr):
    expr = expr.expand()
    coeff_E2 = expr.coeff(E2).expand()
    return (expr - (coeff_E2.coeff(v) * v + coeff_E2.coeff(δv) * δv) * E2).expand()

In [ ]:
res = (E.diff(u) * δu).expand()

We first express the fact that the residual is null along the fundamental branch. In other words,

\begin{equation}
\mathcal{E}_{,u}[u^\star(\lambda), \lambda; \hat{u}] = 0 \quad \text{for all} \quad \hat{u} \in U.
\end{equation}

Plugging the taylor expansion of $u^\star$ delivers expressions of some mixed derivatives of the energy at the critical point.

In [ ]:
mixed_derivatives = dict()
res_star = res.subs(u, u_star).expand()
for k, x in enumerate([E_uλ, E_uλλ, E_uλλλ], start=1):
    eq = res_star.coeff(λ, k)
    sol = solve(eq, x)[0]
    mixed_derivatives[x] = sol

display_dict(mixed_derivatives)

Other mixed derivatives are expressed as derivatives with respect to $\lambda$ of the following differential of the energy, evaluated along the fundamental branch

\begin{equation}
\mathcal{E}_{,uu}[u^\star(\lambda), \lambda; \bullet, \bullet] \quad \text{and} \quad \mathcal{E}_{,uuu}[u^\star(\lambda), \lambda; \bullet, \bullet, \bullet].
\end{equation}

In [ ]:
E2_dot, E2_ddot, E3_dot = symbols("\dot{\E}_2 \ddot{\E}_2 \dot{\E}_3")

E_uu_star = E.diff(u, 2).subs(u, u_star).expand()
E_uuu_star = E.diff(u, 3).subs(u, u_star).expand()

eq = Eq(E2_dot, E_uu_star.coeff(λ, 1))
x = E_uuλ
mixed_derivatives[x] = solve(eq, x)[0]

eq = Eq(E3_dot, E_uuu_star.coeff(λ, 1))
x = E_uuuλ
mixed_derivatives[x] = solve(eq, x)[0]

eq = Eq(E2_ddot, 2 * E_uu_star.coeff(λ, 2))
x = E_uuλλ
mixed_derivatives[E_uuλλ] = solve(eq, x)[0]

display_dict(mixed_derivatives)

And, upon full elimination of the mixed derivatives.

In [ ]:
for x in [E_uλλ, E_uλλλ, E_uuλλ]:
    mixed_derivatives[x] = mixed_derivatives[x].subs(mixed_derivatives).expand()
    
display_dict(mixed_derivatives)

These expressions can be plugged into the expansion of the energy, delivering the following expressions

In [ ]:
E = E.subs(mixed_derivatives).expand()
display(E)

In [ ]:
res = (E.diff(u) * δu).expand()

In [ ]:
ξ, w_ξ, w_λ, w_ξξ, w_ξλ, w_λλ = symbols(r"\xi w_\xi w_\lambda w_{\xi\xi} w_{\xi\lambda} w_{\lambda\lambda}")

w = ξ * w_ξ + λ * w_λ + (ξ**2 * w_ξξ + 2 * ξ * λ * w_ξλ + λ**2 * w_λλ) / 2

In [ ]:
res_w = res.subs({δu: δw, u: u_star + ξ * v + w}).expand()
crit = {ξ: 0, λ: 0}

In [ ]:
eq = Eq(simplify_E2(res_w.diff(ξ)), 0).subs(crit)
display(eq)

In [ ]:
eq = Eq(res_w.diff(λ), 0).subs(crit)
display(eq)

In [ ]:
eq = Eq(res_w.diff(λ, 2), 0).subs(crit).subs(w_λ, 0).expand()
display(eq)

In [ ]:
eq_w_ξξ = Eq(res_w.diff(ξ, 2), 0).subs(crit).subs(w_ξ, 0).expand()
display(eq_w_ξξ)

In [ ]:
eq_w_ξλ = Eq(res_w.diff(ξ, 1, λ, 1), 0).subs(crit).subs({w_ξ: 0, w_λ: 0}).expand()
display(eq_w_ξλ)

## Elimination of $\lambda$

In [ ]:
t = symbols("t")

In [ ]:
w_I, w_II = symbols(r"\xi_kw_{k\lambda}, \xi_i\xi_jw_{ij}")
w_bif = t * λ * w_I + (t * λ)**2 / 2 * w_II
display(w_bif)

In [ ]:
λ_I, λ_II = symbols(r"\xi_{p}\lambda_{p} \xi_m\xi_n\lambda_{mn}")
λ_bif = t * λ_I + t**2 / 2 * λ_II
display(λ_bif)

In [ ]:
ξv = Symbol(r"\xi_{l}v_{l}")
u_bif = u_star + t * ξv + w_bif
display(u_bif)

In [ ]:
res_v = res.subs(δu, δv).subs(u, u_bif).subs(λ, λ_bif).expand()

In [ ]:
assert res_v.coeff(t, 0) == 0

In [ ]:
eqs = [Eq(simplify_E2(res_v.coeff(t, k)), 0, evaluate=False) for k in range(4)]
    
for eq in eqs:
    display(eq)

The first equation should be understood as follows

\begin{equation}
\tfrac{1}{2} \, \xi_n \, \xi_p \, E_3(v_n, v_p, \hat{v}) + \lambda_n \, \xi_n \, \xi_p \, \dot{E}_2(\xi_p, \hat{v}) = 0,
\end{equation}

and, testing with $\hat{v} = v_k$

$$\xi_n \, \xi_p \, E_{npk} + 2\lambda_n \, \xi_n \, \xi_p \, \dot{E}_{pk} = 0.$${#eq-20221221154153}